## Get image/data from OMERO via Blitz Gateway
- Documentation https://docs.openmicroscopy.org/omero/5.6.0/developers/PythonBlitzGateway.html

In [ ]:
import os
import json
import omero.clients
from omero.gateway import BlitzGateway

In [ ]:
thisdir = os.path.abspath('')
interim_data = os.path.join(thisdir, '..\\data\\interim')

#### Create a connection

In [ ]:
server = ""
username = ""
password = ""

In [ ]:
conn = BlitzGateway(username, password, host=server, port=4064)
conn.connect()

In [ ]:
user = conn.getUser()
print("Current user:")
print("   ID:", user.getId())
print("   Username:", user.getName())
print("   Full Name:", user.getFullName())

#### Helper Function

In [ ]:
def print_obj(obj, indent=0):
    """
    Helper method to display info about OMERO objects.
    Not all objects will have a "name" or owner field.
    """
    print("""%s%s:%s  Name:"%s" (owner=%s)""" % (
        " " * indent,
        obj.OMERO_CLASS,
        obj.getId(),
        obj.getName(),
        obj.getOwnerOmeName()))

In [ ]:
my_exp_id = conn.getUser().getId()
default_group_id = conn.getEventContext().groupId

#### Get all image IDs

In [ ]:
img_ids = []
for image in conn.getObjects("Image", opts={'owner': my_exp_id}):
    img_ids.append(image.getId())
img_ids

#### Save all original metadata to json file

In [ ]:
for i in img_ids:
    img = conn.getObject("Image", i)
    meta = img.loadOriginalMetadata()
    f = open(os.path.join(interim_data, "meta_data_"+ img.getName() + ".json"), "w")
    f.write(json.dumps(meta) )
    f.close()

#### Retrieve an image by Image ID

In [ ]:
image = conn.getObject("Image", 7)
print(image.getName(), image.getDescription())

- Retrieve information about an image.

In [ ]:
print(" X:", image.getSizeX())
print(" Y:", image.getSizeY())
print(" Z:", image.getSizeZ())
print(" C:", image.getSizeC())
print(" T:", image.getSizeT())

- List Channels (loads the Rendering settings to get channel colors)

In [ ]:
for channel in image.getChannels():
    print('Channel:', channel.getLabel())
    print('Color:', channel.getColor().getRGB())
    print('Lookup table:', channel.getLut())
    print('Is reverse intensity?', channel.isReverseIntensity())

- render the first timepoint, mid Z section

In [ ]:
z = image.getSizeZ() / 2
t = 0
rendered_image = image.renderImage(z, t)

In [ ]:
rendered_image

- save image

In [ ]:
rendered_image.save("foo.jpg")

In [ ]:
exported_image = image.exportOmeTiff()

#### close connection

In [ ]:
conn.close()